# **Distribución % de créditos por vivienda valor**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "queretaro" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_queretaro',
 'df_financiamientos_2020_queretaro',
 'df_financiamientos_2021_queretaro',
 'df_financiamientos_2022_queretaro',
 'df_financiamientos_2023_queretaro',
 'df_financiamientos_2024_queretaro']

## 2022

In [3]:
finance = df_financiamientos_2022_queretaro[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\1765974849.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... '4' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].astype(str)


,vivienda_valor,conteo
0,-,363
1,Económica,398
2,Media,3307
3,Popular,356
4,Residencial,1821
5,Residencial Plus,589
6,Tradicional,2039


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,3307,0.372704
1,Tradicional,2039,0.229798
2,Residencial,1821,0.205229
3,Residencial Plus,589,0.066381
4,Económica,398,0.044855
5,-,363,0.040911
6,Popular,356,0.040122


In [6]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda.loc[:, 'orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

,vivienda_valor,conteo,promedio
4,Económica,398,0.044855
6,Popular,356,0.040122
1,Tradicional,2039,0.229798
0,Media,3307,0.372704
2,Residencial,1821,0.205229
3,Residencial Plus,589,0.066381


In [7]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [8]:
finance = df_financiamientos_2023_queretaro[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\2884856195.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\2884856195.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\2884856195.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,vivienda_valor,conteo
0,-,500
1,Económica,206
2,Media,3208
3,Popular,306
4,Residencial,1751
5,Residencial Plus,532
6,Tradicional,1861


In [9]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,3208,0.383549
1,Tradicional,1861,0.222501
2,Residencial,1751,0.209350
3,Residencial Plus,532,0.063606
4,-,500,0.059780
5,Popular,306,0.036585
6,Económica,206,0.024629


In [10]:
creditos_vivienda = creditos_vivienda[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda.loc[:, 'orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
6,Económica,206,0.024629
5,Popular,306,0.036585
1,Tradicional,1861,0.222501
0,Media,3208,0.383549
2,Residencial,1751,0.209350
3,Residencial Plus,532,0.063606


In [11]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2023', carpeta='assets\graficas')
fig.show()

## 2024

In [12]:
finance = df_financiamientos_2024_queretaro[['id','vivienda_valor']]
# Convertir la columna 'organismo' a cadena de texto (str)
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].astype(str)
# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'Económica',
    '2': 'Popular',
    '3': 'Tradicional',
    '4': 'Media',
    '5': 'Residencial',
    '6': 'Residencial Plus'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
finance.loc[:, 'vivienda_valor'] = finance['vivienda_valor'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['vivienda_valor'].value_counts().reset_index()
conteo_organismos.columns = ['vivienda_valor', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['vivienda_valor'].unique():
        conteo_organismos = conteo_organismos.append({'vivienda_valor': categoria, 'conteo': 0}, ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='vivienda_valor').reset_index(drop=True)
conteo_organismos

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\3739277305.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\3739277305.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\3739277305.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,vivienda_valor,conteo
0,-,557
1,Económica,23
2,Media,1990
3,Popular,220
4,Residencial,1069
5,Residencial Plus,316
6,Tradicional,1260


In [13]:
# Crear un DataFrame con el conteo de registros por cada organismo
creditos_vivienda = finance['vivienda_valor'].value_counts().reset_index()
creditos_vivienda.columns = ['vivienda_valor', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
creditos_vivienda['promedio'] = creditos_vivienda['conteo'] / total_registros
creditos_vivienda

,vivienda_valor,conteo,promedio
0,Media,1990,0.366145
1,Tradicional,1260,0.231831
2,Residencial,1069,0.196688
3,-,557,0.102484
4,Residencial Plus,316,0.058142
5,Popular,220,0.040478
6,Económica,23,0.004232


In [14]:
creditos_vivienda = creditos_vivienda.loc[creditos_vivienda['vivienda_valor'] != '-']

# Definir el diccionario de orden de organismos
orden_viv = {
    'Económica':1, 
    'Popular':2,
    'Tradicional':3,
    'Media':4,
    'Residencial':5,
    'Residencial Plus':6
}

creditos_vivienda.loc[:, 'orden'] = creditos_vivienda['vivienda_valor'].map(orden_viv)
# Ordenar el DataFrame por el orden de los organismos
creditos_vivienda = creditos_vivienda.sort_values(by='orden')
creditos_vivienda = creditos_vivienda.drop(columns=['orden'])
creditos_vivienda

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4056\531983862.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,vivienda_valor,conteo,promedio
6,Económica,23,0.004232
5,Popular,220,0.040478
1,Tradicional,1260,0.231831
0,Media,1990,0.366145
2,Residencial,1069,0.196688
4,Residencial Plus,316,0.058142


In [15]:
labels = creditos_vivienda['vivienda_valor']
values = creditos_vivienda['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por vivienda valor (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_vivienda_2024', carpeta='assets\graficas')
fig.show()